In [28]:
from sklearn.ensemble import VotingClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score,accuracy_score,log_loss,r2_score
from sklearn.model_selection import train_test_split,StratifiedKFold , GridSearchCV
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer ,make_column_selector
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

In [29]:
hr = pd.read_csv('HR_comma_sep.csv')

In [30]:
X = hr.drop('left',axis=1)
y = hr['left']

In [40]:
X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

In [32]:
# ohe = OneHotEncoder(handle_unknown = 'ignore')
# ct = make_column_transformer((ohe,make_column_selector(dtype_include=object)) ,
#                                    ('passthrough',make_column_selector(dtype_exclude=object)),
#                                    verbose_feature_names_out=False)
# dtc = DecisionTreeClassifier(random_state=24,max_depth=3)
# lr = LogisticRegression(random_state=24)
# nb = GaussianNB()
# kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

# voting = VotingClassifier([('DT',dtc),('LR',lr),('NB',nb)])
# pipe = Pipeline([('CT',ct),('VOTING',voting)])
# pipe.get_params()
# params = {'VOTING__DT__max_depth':[None,2,3],
#          'VOTING__DT__min_samples_leaf':[1,10,20],
#          'VOTING__DT__min_samples_split':[2,10,20],
#          'VOTING__LR__C':np.linspace(0.001,3,5)}
# gcv = GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='roc_auc',verbose=3)
# gcv.fit(X,y)

#### CatBoost without One Hot Encoding

In [41]:
from catboost import CatBoostClassifier
cgbm = CatBoostClassifier(random_state=24,cat_features=['Department','salary'],n_estimators=50)
cgbm.fit(X_train,y_train)

Learning rate set to 0.438486
0:	learn: 0.4127012	total: 53.2ms	remaining: 2.6s
1:	learn: 0.2883445	total: 94.4ms	remaining: 2.26s
2:	learn: 0.2171350	total: 133ms	remaining: 2.09s
3:	learn: 0.1787252	total: 176ms	remaining: 2.02s
4:	learn: 0.1554308	total: 216ms	remaining: 1.95s
5:	learn: 0.1402544	total: 270ms	remaining: 1.98s
6:	learn: 0.1274649	total: 310ms	remaining: 1.9s
7:	learn: 0.1223568	total: 348ms	remaining: 1.83s
8:	learn: 0.1151863	total: 393ms	remaining: 1.79s
9:	learn: 0.1090791	total: 436ms	remaining: 1.75s
10:	learn: 0.1055714	total: 478ms	remaining: 1.69s
11:	learn: 0.1027904	total: 538ms	remaining: 1.7s
12:	learn: 0.1001674	total: 581ms	remaining: 1.65s
13:	learn: 0.0980211	total: 619ms	remaining: 1.59s
14:	learn: 0.0960815	total: 668ms	remaining: 1.56s
15:	learn: 0.0946345	total: 723ms	remaining: 1.53s
16:	learn: 0.0927250	total: 762ms	remaining: 1.48s
17:	learn: 0.0906349	total: 815ms	remaining: 1.45s
18:	learn: 0.0885028	total: 861ms	remaining: 1.4s
19:	learn: 0.

In [42]:
y_pred = cgbm.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.9773282951767059


In [45]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [43]:

kfold = StratifiedKFold(n_splits = 5,shuffle=True,random_state=24)
params = {'n_estimators':[10,50],
         'max_depth':[2,3,4],
         'learning_rate':np.linspace(0.001,1,5)}
gcv = GridSearchCV(cgbm, param_grid=params, cv=kfold, scoring='roc_auc', verbose=3)
gcv.fit(X,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
0:	learn: 0.6925472	total: 13.6ms	remaining: 122ms
1:	learn: 0.6919348	total: 29.8ms	remaining: 119ms
2:	learn: 0.6913254	total: 52.2ms	remaining: 122ms
3:	learn: 0.6907172	total: 63.7ms	remaining: 95.6ms
4:	learn: 0.6901085	total: 74.7ms	remaining: 74.7ms
5:	learn: 0.6895010	total: 89ms	remaining: 59.4ms
6:	learn: 0.6888947	total: 104ms	remaining: 44.5ms
7:	learn: 0.6882913	total: 125ms	remaining: 31.2ms
8:	learn: 0.6876875	total: 140ms	remaining: 15.6ms
9:	learn: 0.6870981	total: 157ms	remaining: 0us
[CV 1/5] END learning_rate=0.001, max_depth=2, n_estimators=10;, score=0.911 total time=   0.1s
0:	learn: 0.6925453	total: 12.3ms	remaining: 111ms
1:	learn: 0.6919295	total: 36.2ms	remaining: 145ms
2:	learn: 0.6913171	total: 58.2ms	remaining: 136ms
3:	learn: 0.6907059	total: 72.2ms	remaining: 108ms
4:	learn: 0.6900937	total: 83.7ms	remaining: 83.7ms
5:	learn: 0.6894828	total: 98.5ms	remaining: 65.7ms
6:	learn: 0.6888731	total:

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x000002037EF52EA0>,
             param_grid={'learning_rate': array([0.001  , 0.25075, 0.5005 , 0.75025, 1.     ]),
                         'max_depth': [2, 3, 4], 'n_estimators': [10, 50]},
             scoring='roc_auc', verbose=3)

In [44]:
print(gcv.best_params_)
print(gcv.best_score_)

{'learning_rate': 1.0, 'max_depth': 4, 'n_estimators': 50}
0.9893002633546979
